# Common errors and possible solutions

## Building STM32F3 firmware

When building STM32F3 firmware, depending on which version of chipwhisperer you are using, you may get the error message;
```

Compiling:
-en …/./hal/stm32f3/stm32f3_sysmem.c …
…/./hal/stm32f3/stm32f3_sysmem.c:63:1: error: unknown type name ‘caddr_t’
63 | caddr_t _sbrk(int incr)
| ^~~~~~~
…/./hal/stm32f3/stm32f3_sysmem.c: In function ‘_sbrk’:
…/./hal/stm32f3/stm32f3_sysmem.c:76:25: error: ‘caddr_t’ undeclared (first use in this function)
76 | return (caddr_t) -1;
| ^~~~~~~
…/./hal/stm32f3/stm32f3_sysmem.c:76:25: note: each undeclared identifier is reported only once for each function it appears in
…/./hal/stm32f3/stm32f3_sysmem.c:81:25: error: expected ‘;’ before ‘prev_heap_end’
81 | return (caddr_t) prev_heap_end;
...

CalledProcessError: Command ‘b’cd 
/Users/Thomas/Downloads/chipwhisperer-5.7.0/hardware/victims/firmware/simpleserial-glitch\ngmake 
PLATFORM=$1 CRYPTO_TARGET=NONE SS_VER=$2 -j\n’’ returned non-zero exit status 2`
```
This may be because of the fact that some ```arm-none-eab-gcc``` compilers doesn not suppert ```caddr_t``` which is a function in STM32F3_sysmem.c. To fix this just update Chipwhisperer via git 
```
cd ..
git submodule init jupyter/
git submodule update
```
or just change the function locally with this https://github.com/newaetech/chipwhisperer/blob/develop/hardware/victims/firmware/hal/stm32f3/stm32f3_sysmem.c 

## No trigger seen and Device did not ack

 When trying to capture traces you may experience errors/warnings such as ```No trigger seen``` and ```Device did not ack```. This is due to the fact that the capture device can not find the target. ```No trigger seen``` is often an error that occurs when we are changing the clock (i.e ```scope.clock.adc_src = "clkgen_x1"``` -->```scope.clock.adc_src = "clkgen_x4```)  The usual solution to this problem is to reset the target with the function ```reset_target(scope)```. You can also try to reprogram firmware or changing simpleserial version.

### Chipwisperer analyzer

The problem can also occur when working with chipwhisperer analyzer for example in Lab 4_3 - ChipWhisperer Analyzer CPA Attack then a quick fix is changing the project from ```cw.open_project()``` to ```cw.create_project()```.

## LIBUSB_ERROR_ACCESS [-3]

This is a really common error with a quick fix. It occurs because Chipwhisperer can not run on two kernels at the same time and to fix it just run 
```
scope.dis()
target.dis()
```
or restart the kernel that you are not using.

## SerializationError: can’t serialize <class ‘range’>

This error often occured when using Bokehjs and Holoviews. There is an easy fix to this and we just need to make a list of the range object in use
i.e 

```xrange = range(len(trace_array[0]))[plot_start:plot_end]```
--> 

```xrange = list(range(len(trace_array[0]))[plot_start:plot_end])```


## scope.io.vglitch_reset()

The function ```scope.io.vglitch_reset``` is used in capturing voltage glitches in the labs revolving around fault injections, but it does not exist in the newest software release. This is a fairly new function and it is added in the development branch on github. To bypass the problem either comment out the function from your cell in jupyter, update via git or add it in ChipwhispererExtra.py as they have done here https://github.com/newaetech/chipwhisperer/commit/f922eb99c8ad84f1586768cc7bbb39e5c55c0162

## gc.results.calc()

We encounter a similar issue as above with the argument sort in gc.results.calc() as it does not exist in the latest release. The fix is to either add it to glitch.py, update via git or just ignore if the function can  sort your parameters or not. If you want to add it, follow the same link as above. They were added at the same time.

## ModuleError: no module named Crypto

Due to some issues pycrypto is no longer used and you should use pycryptodome instead.

to solve this run;
```
pip uninstall crypto 
pip uninstall pycrypto 
pip install pycryptodome
```


https://stackoverflow.com/questions/19623267/importerror-no-module-named-crypto-cipher